In [91]:
%load_ext autoreload
%autoreload 2

import os,sys
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

import util as yu
from util import *
import util_NpiScatteringWilson as yun

yu.setpath('analysis')

ens='milc'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
def symmetrize(dat):
    dat_bw=np.roll(np.flip(dat,axis=1),1,axis=1)
    return((dat[:,:,0]+dat[:,:,5]) - (dat_bw[:,:,10]+dat_bw[:,:,15]))/4

d_jk=3
Nsrc=100
path=f'/capstor/store/cscs/userlab/lp139/lyan/code/projectData/03_NpiScatteringWilson/MILC_a09m130W/data_merge/NpiScatteringWilson_Nsrc={Nsrc}.h5'
data={}
with h5py.File(path) as f:    
    cfgs_all=[cfg.decode() for cfg in f['cfgs'][:]]
    # cfgs_map=[cfgs_all.index(cfg) for cfg in cfgs]
    
    if True:
        cfgs_map=[cfgs_all.index(cfg) for cfg in cfgs_all]
    
    for cont in f['data'].keys():
        if cont in ['N']:
            t=f[f'data/{cont}'][:,:,16]
            t=symmetrize(t)
            t=np.real(t)
            data[cont]=yu.jackknife(t[cfgs_map],d=d_jk)
            continue
        if cont in ['P']:
            t=f[f'data/{cont}'][:,:,0]
            t_bw=np.roll(np.flip(t,axis=1),1,axis=1)
            t=(t+t_bw)/2
            t=np.real(t)
            data[cont]=yu.jackknife(t[cfgs_map],d=d_jk)
            continue
        t=f[f'data/{cont}'][:,:,0]
        t=symmetrize(t)
        t=np.real(t)
        data[cont]=yu.jackknife(t[cfgs_map],d=d_jk)

Njk=len(data['N'])
print(data['N'].shape)

path='pkl/analysis_c2pt_pi_N/reg_ignore/mpi.pkl'
mpi_jk=yu.load_pkl(path)

path='pkl/analysis_c2pt_pi_N/reg_ignore/pars_jk_meffnst_selected.pkl'
[pars_jk_meff1st,pars_jk_meff2st,pars_jk_meff3st]=yu.load_pkl(path)

mN_jk=pars_jk_meff2st[:,0]

(113, 96)


In [5]:
coeffs_32=[[[1,'B232'],[-1,'B122']],[[1,'W1222'],[-1,'W1321'],[1,'W1322'],[-1,'W2321']],[[-1,'Z114'],[1,'Z132'],[-1,'Z214'],[-1,'Z222']]]
coeffs_12=[[[-3/2,'B114'],[1/2,'B122'],[3/2,'B132'],[-3/2,'B214'],[1,'B232']],
           [[-3/2,'W1141'],[3/2,'W1142'],[3/2,'W1221'],[-1/2,'W1222'],[1/2,'W1321'],[-1/2,'W1322'],[-1,'W2321']],
           [[1/2,'Z114'],[-3/2,'Z122'],[-1/2,'Z132'],[1/2,'Z214'],[-1,'Z222']]
]
method=['2','2','1']

Nstoc=12

c_12=np.real(data['M'][:,:,0]+np.sum([
    np.mean(cont[0]*data[f'{cont[1]}_{method}'][:,:,:] if cont[1][0] in [] else cont[0]*data[f'{cont[1]}_{method}'][:,:,:Nstoc],axis=-1)
    for conts,method in zip(coeffs_12,method) for cont in conts],axis=0))
c_32=np.real(data['M'][:,:,0]+np.sum([
    np.mean(cont[0]*data[f'{cont[1]}_{method}'][:,:,:] if cont[1][0] in [] else cont[0]*data[f'{cont[1]}_{method}'][:,:,:Nstoc],axis=-1)
    for conts,method in zip(coeffs_32,method) for cont in conts],axis=0))
c_Npi=np.real(data['N']*data['P'])
c_N=np.real(data['N'])
c_pi=np.real(data['P'])

isos=['32','12']
iso2c={'32':c_32,'12':c_12}

In [145]:
mu_jk=1/(1/mpi_jk+1/mN_jk)
iso2a0mpi_phy={iso:yu.jackknife_pseudo(*yun.iso2a0mpi_phy_me[iso],Njk)[:,0] for iso in isos}
iso2adE_phy={iso:np.array([yun.sl2es(mu,yun.ens2NL[ens],a0mpi/ampi) for mu,a0mpi,ampi in zip(mu_jk,iso2a0mpi_phy[iso],mpi_jk)]) for iso in isos}

iso2a0mpi_pol={iso:yu.jackknife_pseudo(*yun.iso2a0mpi_pol_me[iso],Njk)[:,0] for iso in isos}
iso2adE_pol={iso:np.array([yun.sl2es(mu,yun.ens2NL[ens],a0mpi/ampi) for mu,a0mpi,ampi in zip(mu_jk,iso2a0mpi_pol[iso],mpi_jk)]) for iso in isos}

iso2a0mpi_delta={iso:yu.jackknife_pseudo(*yun.iso2a0mpi_delta_me[iso],Njk)[:,0] for iso in isos[:1]}
iso2adE_delta={iso:np.array([yun.sl2es(mu,yun.ens2NL[ens],a0mpi/ampi) for mu,a0mpi,ampi in zip(mu_jk,iso2a0mpi_delta[iso],mpi_jk)]) for iso in isos[:1]}

for iso in isos:
    print(f'dE^{iso}:',yu.jackme_un2str(iso2adE_phy[iso]*yun.ens2aInv[ens])+';',yu.jackme_un2str(iso2adE_pol[iso]*yun.ens2aInv[ens])+';',yu.jackme_un2str(iso2adE_delta[iso]*yun.ens2aInv[ens])+';'  if iso in iso2adE_delta else ' - ;')
    
    
colors_phy=['r','b']
colors_pol=['darkred','darkblue']
colors_delta=['green']

dE^32: 1.496(33); 1.83(34); 2.33(78);
dE^12: -2.460(28); -2.29(41);  - ;


In [147]:
fig,axs=yu.getFigAxs(1,1)
ax=axs[0,0]
xunit=yun.ens2a[ens]; yunit=yun.ens2aInv[ens]
ax.set_ylim([-5,10])

colors=['r','b']
for iiso,iso in enumerate(isos):
    c=iso2c[iso]/c_Npi
    dt=1
    func=lambda c2pt:np.log(c2pt/np.roll(c2pt,-dt,axis=0))/dt
    dm=yu.jackmap(func,c)
    ts=np.arange(20)
    
    mean,err=yu.jackme(dm)
    plt_x=ts*xunit; plt_y=mean[ts]*yunit; plt_yerr=err[ts]*yunit
    ax.errorbar(plt_x,plt_y,plt_yerr,color=colors[iiso])
    
    if iso in ['32']:
        mean,err=yu.jackme(iso2adE_delta[iso])
        plt_y=mean*yunit; plt_yerr=err*yunit
        ax.axhspan(plt_y-plt_yerr,plt_y+plt_yerr,color=colors_delta[iiso],alpha=0.2)
        
    mean,err=yu.jackme(iso2adE_pol[iso])
    plt_y=mean*yunit; plt_yerr=err*yunit
    ax.axhspan(plt_y-plt_yerr,plt_y+plt_yerr,color=colors_pol[iiso],alpha=0.2)
    
    mean,err=yu.jackme(iso2adE_phy[iso])
    plt_y=mean*yunit; plt_yerr=err*yunit
    ax.axhspan(plt_y-plt_yerr,plt_y+plt_yerr,color=colors_phy[iiso],alpha=0.2)
    
yu.finalizePlot('dE_eff')

In [151]:
fig,axs=yu.getFigAxs(1,3)
colors=['r','b']

for iiso,iso in enumerate(isos):
    ax=axs[0,0]
    ax.set_xlabel(r'$t$')
    ax.set_ylabel(r'$C_{N\pi}/[C_{N}\,C_{\pi}]$')
    xunit=yun.ens2a[ens]; yunit=1
    ax.set_ylim([0.95,1.02])
    yu.addRefLine(ax,1)
    
    
    c=iso2c[iso]/c_Npi
    ts=np.arange(1,20)
    
    mean,err=yu.jackme(c)
    plt_x=ts*xunit; plt_y=mean[ts]*yunit; plt_yerr=err[ts]*yunit
    ax.errorbar(plt_x,plt_y,plt_yerr,color=colors[iiso])
    
    slope=-np.mean(iso2adE_phy[iso])
    t0=12; y0=mean[t0]
    ys=slope*(ts-t0) + y0
    ax.plot(ts*xunit,ys,color=colors[iiso],ls='--',marker='')

    ax=axs[0,1]
    ax.set_xlabel(r'$t_{\rm min}$')
    ax.set_ylabel(r'$\Delta E$')
    xunit=yun.ens2a[ens]; yunit=yun.ens2aInv[ens]
    ax.set_ylim([-5,10])   
    yu.addRefLine(ax,0)

    # if iso in ['32']:
    #     mean,err=yu.jackme(iso2adE_delta[iso])
    #     plt_y=mean*yunit; plt_yerr=err*yunit
    #     ax.axhspan(plt_y-plt_yerr,plt_y+plt_yerr,color=colors_delta[iiso],alpha=0.2)
        
    mean,err=yu.jackme(iso2adE_pol[iso])
    plt_y=mean*yunit; plt_yerr=err*yunit
    ax.axhspan(plt_y-plt_yerr,plt_y+plt_yerr,color=colors_pol[iiso],alpha=0.2)
    
    mean,err=yu.jackme(iso2adE_phy[iso])
    plt_y=mean*yunit; plt_yerr=err*yunit
    ax.axhspan(plt_y-plt_yerr,plt_y+plt_yerr,color=colors_phy[iiso],alpha=0.2)
    
    y_jk=iso2c[iso]/c_Npi
    ts=np.arange(len(y_jk[0]))
    
    tmins=np.arange(1,16); tmaxs=[25]
    fits=yu.doFits_linear(ts,y_jk,tmins,tmaxs,corrQ=False)
    for fit in fits:
        (tmin,tmax),pars_jk,chi2_jk,Ndof=fit
        mean,err=yu.jackme(-pars_jk[:,1])
        plt_x=(tmin+0.2*tmaxs.index(tmax))*xunit; plt_y=mean*yunit; plt_yerr=err*yunit
        ax.errorbar(plt_x,plt_y,plt_yerr,color=colors[iiso])
        
    ax=axs[0,2]
    ax.set_xlabel(r'$t_{\rm min}$')
    ax.set_ylabel(r'$a_0\,m_\pi$')
    xunit=yun.ens2a[ens]; yunit=1
    ax.set_ylim([-0.5*yunit,0.5*yunit])   
    yu.addRefLine(ax,0)
    
    mean,err=yu.jackme(iso2a0mpi_pol[iso])
    plt_y=mean*yunit; plt_yerr=err*yunit
    ax.axhspan(plt_y-plt_yerr,plt_y+plt_yerr,color=colors_pol[iiso],alpha=0.2)
    
    mean,err=yu.jackme(iso2a0mpi_phy[iso])
    plt_y=mean*yunit; plt_yerr=err*yunit
    ax.axhspan(plt_y-plt_yerr,plt_y+plt_yerr,color=colors_phy[iiso],alpha=0.2)
    
    for fit in fits:
        (tmin,tmax),pars_jk,chi2_jk,Ndof=fit
        t=[yun.es2sl(mu,yun.ens2NL[ens],dE)*ampi for mu,dE,ampi in zip(mu_jk,-pars_jk[:,1],mpi_jk)]
        mean,err=yu.jackme(t)
        plt_x=(tmin+0.2*tmaxs.index(tmax))*xunit; plt_y=mean*yunit; plt_yerr=err*yunit
        ax.errorbar(plt_x,plt_y,plt_yerr,color=colors[iiso])

yu.finalizePlot('ratio_DE_a0mpi')